### GitHub repository
Link to repository used to colaborate on the assignment:
https://github.com/KarolineKlan/Assignments_ComSocSci2024.git

### Contribution statement

Team members:

- Jacob (s214596)
- Kristoffer (s214609)
- Karoline (s214638)

All members collaborated and contributed to every part of the assignment.


# Assignment 1
This assignment was formed using Web-scraping tools from the program of the International Conference in Computational Social Science 2023  https://ic2s2-2023.org/program, and acessing data of Authors and Research Articles using the OpenAlex API https://docs.openalex.org/.

In [2]:
#Import relevant libraries
from bs4 import BeautifulSoup 
import requests
import pandas as pd
from tqdm import tqdm
import Levenshtein
import numpy as np
import ast
import networkx as nx

## Part 1 - Webscraping
In the following task we use web-scraping tools to get the list of participants in the International Conference in Computational Social Science 2023


**The process of the web-scraping:** 

Answer

## Part 2 - Ready Made vs Custom Made Data

Write

## Part 3 - Gathering Research Articles using the OpenAlex API

### Data Overview and Reflection questions: 

- **Dataset summary.** 

- **Efficiency in code.** 


- **Filtering Criteria and Dataset Relevance**

## Part 4: The Network of Computational Social Scientists

In this part of the assignment we construct and investigate the Computational Social Scientists Network.